In [ ]:
#pip install keras_tuner

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import matplotlib.pyplot as plt
import sklearn as skl
from sklearn.datasets import make_moons
from tabulate import tabulate


In [ ]:
# Import pandas and read the charity_data.csv from the provided cloud URL.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop non-beneficial columns
application_df = application_df.drop(columns=["EIN", "NAME"])


In [ ]:
# Check the value counts for APPLICATION_TYPE
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Bin rare classifications
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

In [ ]:
# Convert categorical variables to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [ ]:
# Split the data into features and target
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values
y = application_df["IS_SUCCESSFUL"].values


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Step 2: Define and optimize the neural network
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

    # Allow kerastuner to decide number of neurons in the first layer
    nn_model.add(tf.keras.layers.Dense(
        units=hp.Int('first_units', min_value=10, max_value=100, step=10),
        activation=activation,
        input_dim=X_train_scaled.shape[1]
    ))

    # Allow kerastuner to decide number of hidden layers and their neurons
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(
            units=hp.Int('units_' + str(i), min_value=10, max_value=100, step=10),
            activation=activation
        ))

    # Output layer
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model


In [ ]:
# Import the kerastuner library
tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=50,
    factor=3,
    directory='tuner_results',
    project_name='charity_optimization_tuning'
)

tuner.search(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test))

Trial 84 Complete [00h 02m 31s]
val_accuracy: 0.7266764044761658

Best val_accuracy So Far: 0.7300291657447815
Total elapsed time: 00h 52m 42s


In [ ]:
# Retrieve the best model
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]


In [ ]:
# Train the model with increased epochs
history = best_model.fit(X_train_scaled, y_train, epochs=150, batch_size=32, verbose=1)


Epoch 1/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7300 - loss: 0.5499
Epoch 2/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7327 - loss: 0.5475
Epoch 3/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7308 - loss: 0.5481
Epoch 4/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7324 - loss: 0.5468
Epoch 5/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7332 - loss: 0.5484
Epoch 6/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7341 - loss: 0.5440
Epoch 7/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7330 - loss: 0.5460
Epoch 8/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7340 - loss: 0.5458
Epoch 9/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7366 - loss: 0.5438
Epoch 10/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7338 - loss: 0.5437
Epoch 11/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7354 - loss: 0.5475
Epoch 12/150
858/858 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Evaluate the optimized model
model_loss, model_accuracy = best_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Optimized Loss: {model_loss}, Optimized Accuracy: {model_accuracy}")

215/215 - 1s - 4ms/step - accuracy: 0.7239 - loss: 0.5600
Optimized Loss: 0.5600075125694275, Optimized Accuracy: 0.7239066958427429


In [ ]:
# Save the best model
best_model.save("AlphabetSoupCharity_Optimization_Tuned.h5")
print("\nOptimized model saved as 'AlphabetSoupCharity_Optimization_Tuned.h5'.")



Optimized model saved as 'AlphabetSoupCharity_Optimization_Tuned.h5'.
